In [2]:
# import useful libraries
import re
import datetime
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import matplotlib.pyplot as plt
pd.set_option("display.max_columns",5000)

In [3]:
# read datasets
def get_income_data(diff=1, date_range=14):
    df = []
    for i in range(date_range):
        date = (datetime.datetime.today()-datetime.timedelta(diff) - datetime.timedelta(1+i)).strftime("%Y%m%d")
        try:
            data = pd.read_parquet(f"../data_rr/vod_ldd_ecpm_daily_details_rr/year=2023/month={int(date[4:6])}/day={int(date[6:8])}")
            df.append(data)
        except:
            print('path not exists',f"../data_rr/vod_ldd_ecpm_daily_details_rr/year=2023/month={int(date[4:6])}/day={int(date[6:8])}")
    df = pd.concat(df)
    return df

def get_exposure_data(diff=1, date_range=14):
    from vod_metrics import LddBaseCidMetricsStore
    ldd_metrics = LddBaseCidMetricsStore(
        'ldd', 
        datetime.date.today() - datetime.timedelta(days=date_range),
        datetime.date.today() - datetime.timedelta(days=diff),
    )
    trex_14days = ldd_metrics.trex_metrics
    return trex_14days

def get_vip_data(diff=1, date_range=14):
    data = pd.read_parquet(f"../data_rr/vod_ldd_ecpm_vip_details_rr")
    start_date = (datetime.datetime.today() - datetime.timedelta(date_range)).strftime("%Y%m%d")
    start = str(start_date[:4])+'-'+str(start_date[4:6])+'-'+str(start_date[6:8])
    end_date = datetime.datetime.today().strftime("%Y%m%d")
    end = str(end_date[:4])+'-'+str(end_date[4:6])+'-'+str(end_date[6:8])
    start_date = pd.to_datetime(start)
    end_date = pd.to_datetime(end)
    is_categorical = isinstance(data['data_date'].dtype, pd.CategoricalDtype)
    if is_categorical:
        data['data_date'] = data['data_date'].astype(str)
        data['data_date'] = pd.to_datetime(data['data_date'])
    filtered_data_categorical = data[(data['data_date'] >= start_date) & (data['data_date'] <= end_date)]
    return filtered_data_categorical


ldd_media = pd.read_parquet("../data_rr/ldd_media/child_tags_20220215.parquet")
vip_14days = get_vip_data(diff=1, date_range=14) # 过去14天的vip数据
income_14days = get_income_data(diff=1, date_range=14) # 过去14天的income数据

In [4]:
# merge datasets
df = income_14days.merge(vip_14days, on=['baseCid', 'cid', 'dnum', 'algo_type', 'pay_amt', 'sd_income'], how='left')
df = df[['baseCid', 'cid', 'pay_amt', 'sd_income','buy_times','duration_type']]
df = df.merge(ldd_media[['cid', 'ip']], on='cid', how='left', suffixes=('_of_cid', '_of_cid'))
df = df.merge(ldd_media[['cid', 'ip']], left_on='baseCid', right_on='cid', how='left', suffixes=('', '_of_baseCid')).drop('cid_of_baseCid', axis=1)

In [5]:
df

,baseCid,cid,pay_amt,sd_income,buy_times,duration_type,ip,ip_of_baseCid
0,JIRfwLGdYA1619589353988,yFumQprtAi1624962424608,28.00,16.8000,2.0,月卡,汽车世界,汽车世界
1,RYkFLhqgAh1608551836986,WGDvBHqoSK1660728042724,59.00,59.0000,3.0,季卡,NaN,宝宝巴士
2,ApQkswPzZM1654872253896,ljBQVzHtwm1604037148198,28.00,28.0000,2.0,月卡,汽车世界,NaN
3,ljBQVzHtwm1604037148198,BGpgEjjKgS1678844135010,1.00,1.0000,1.0,周卡,NaN,汽车世界
4,NLPhMBKtvg1669646035008,yFumQprtAi1624962424608,28.00,16.8000,2.0,月卡,汽车世界,NaN
...,...,...,...,...,...,...,...,...
2567,hONzltnDij1635755555937,JzRkcivWuU1647420472731,55.00,55.0000,NaN,NaN,NaN,宝宝巴士
2568,yyNhGxSAqI1645438099053,WlJpMZIkWj1660728776348,19.00,19.0000,NaN,NaN,NaN,NaN
2569,qJOsiIczzB1603373597958,BebfPMtgEu1593583439097,28.00,28.0000,NaN,NaN,碰碰狐,碰碰狐
2570,vyouLBZrvX1593583439097,BebfPMtgEu1593583439097,1.00,1.0000,NaN,NaN,碰碰狐,碰碰狐


In [6]:
df['ip_of_baseCid'] = df.apply(lambda row: row['ip_of_baseCid'] if row['ip_of_baseCid'] == row['ip'] else f'not-{row["ip"]}', axis=1)
grouped_transformed_df = df.groupby(['ip', 'ip_of_baseCid', 'duration_type']).agg({
    'pay_amt': 'sum',
    'sd_income': 'sum',
    'buy_times': 'sum'
}).reset_index()
grouped_transformed_df

,ip,ip_of_baseCid,duration_type,pay_amt,sd_income,buy_times
0,23号牛乃唐,23号牛乃唐,月卡,28.00,28.0000,5.0
1,kiki玩具,kiki玩具,周卡,1.00,1.0000,1.0
2,亲宝,亲宝,月卡,38.00,38.0000,14.0
3,会说话的汤姆猫,not-会说话的汤姆猫,周卡,1.00,0.5000,1.0
4,会说话的汤姆猫,not-会说话的汤姆猫,季卡,177.00,177.0000,6.0
...,...,...,...,...,...,...
206,道奇,not-道奇,季卡,55.00,55.0000,8.0
207,钢铁飞龙,not-钢铁飞龙,月卡,28.00,14.0000,1.0
208,阿衰,阿衰,月卡,28.00,28.0000,2.0
209,魔仙俏娇娃,not-魔仙俏娇娃,月卡,19.00,19.0000,1.0


In [7]:
filtered_df = grouped_transformed_df[grouped_transformed_df['ip'] == grouped_transformed_df['ip_of_baseCid']]
filtered_df

,ip,ip_of_baseCid,duration_type,pay_amt,sd_income,buy_times
0,23号牛乃唐,23号牛乃唐,月卡,28.00,28.0000,5.0
1,kiki玩具,kiki玩具,周卡,1.00,1.0000,1.0
2,亲宝,亲宝,月卡,38.00,38.0000,14.0
6,会说话的汤姆猫,会说话的汤姆猫,季卡,89.00,44.5000,1.0
7,会说话的汤姆猫,会说话的汤姆猫,月卡,6.60,3.3000,1.0
...,...,...,...,...,...,...
202,迷你特工队,迷你特工队,周卡,4.00,4.0000,16.0
203,迷你特工队,迷你特工队,季卡,55.00,55.0000,18.0
204,迷你特工队,迷你特工队,年卡,229.00,229.0000,2.0
205,迷你特工队,迷你特工队,月卡,122.90,67.7300,11.0


In [9]:
pivot_table = filtered_df.pivot_table(values='sd_income', index=['ip', 'ip_of_baseCid'], columns='duration_type', aggfunc='sum').reset_index()
pivot_table['total_sd_income'] = pivot_table.drop(columns=['ip', 'ip_of_baseCid']).sum(axis=1)
for col in pivot_table.columns:
    if col not in ['ip', 'ip_of_baseCid', 'total_sd_income']:
        pivot_table[col] = pivot_table[col].astype('float64')
        pivot_table[col] = pivot_table[col] / pivot_table['total_sd_income']
pivot_table = pivot_table.drop(columns='total_sd_income')
pivot_table = pivot_table.fillna(0)
pivot_table

duration_type,ip,ip_of_baseCid,半年卡,周卡,季卡,年卡,月卡
0,23号牛乃唐,23号牛乃唐,0.00000,0.000000,0.000000,0.000000,1.000000
1,kiki玩具,kiki玩具,0.00000,1.000000,0.000000,0.000000,0.000000
2,亲宝,亲宝,0.00000,0.000000,0.000000,0.000000,1.000000
3,会说话的汤姆猫,会说话的汤姆猫,0.00000,0.000000,0.930962,0.000000,0.069038
4,兔小贝,兔小贝,0.00000,0.050000,0.000000,0.000000,0.950000
5,全能宝贝,全能宝贝,0.00000,1.000000,0.000000,0.000000,0.000000
6,变形警车珀利,变形警车珀利,0.00000,1.000000,0.000000,0.000000,0.000000
7,名侦探柯南,名侦探柯南,0.00000,1.000000,0.000000,0.000000,0.000000
8,咕力咕力,咕力咕力,0.00000,0.000000,0.000000,0.000000,1.000000
9,咚咚玩具屋,咚咚玩具屋,0.00000,1.000000,0.000000,0.000000,0.000000


In [10]:
pivot_table['same_ip'] = (pivot_table['ip'] == pivot_table['ip_of_baseCid']).astype(int)
grouped_df = pivot_table.groupby('same_ip').mean().reset_index()
grouped_df

duration_type,same_ip,半年卡,周卡,季卡,年卡,月卡
0,1,0.002143,0.231584,0.18542,0.058248,0.522607


In [11]:
filtered_df2 = grouped_transformed_df[grouped_transformed_df['ip'] != grouped_transformed_df['ip_of_baseCid']]
filtered_df2

,ip,ip_of_baseCid,duration_type,pay_amt,sd_income,buy_times
3,会说话的汤姆猫,not-会说话的汤姆猫,周卡,1.00,0.5000,1.0
4,会说话的汤姆猫,not-会说话的汤姆猫,季卡,177.00,177.0000,6.0
5,会说话的汤姆猫,not-会说话的汤姆猫,月卡,62.60,59.3000,17.0
8,倒霉蛋涂涂,not-倒霉蛋涂涂,周卡,2.00,2.0000,3.0
9,僵小鱼,not-僵小鱼,周卡,1.00,1.0000,1.0
...,...,...,...,...,...,...
201,迷你特工队,not-迷你特工队,月卡,126.00,71.7100,9.0
206,道奇,not-道奇,季卡,55.00,55.0000,8.0
207,钢铁飞龙,not-钢铁飞龙,月卡,28.00,14.0000,1.0
209,魔仙俏娇娃,not-魔仙俏娇娃,月卡,19.00,19.0000,1.0


In [12]:
pivot_table2 = filtered_df2.pivot_table(values='sd_income', index=['ip', 'ip_of_baseCid'], columns='duration_type', aggfunc='sum').reset_index()
pivot_table2['total_sd_income'] = pivot_table2.drop(columns=['ip', 'ip_of_baseCid']).sum(axis=1)
for col in pivot_table2.columns:
    if col not in ['ip', 'ip_of_baseCid', 'total_sd_income']:
        pivot_table2[col] = pivot_table2[col].astype('float64')
        pivot_table2[col] = pivot_table2[col] / pivot_table2['total_sd_income']
pivot_table2 = pivot_table2.drop(columns='total_sd_income')
pivot_table2 = pivot_table2.fillna(0)
pivot_table2

duration_type,ip,ip_of_baseCid,半年卡,周卡,季卡,年卡,月卡
0,会说话的汤姆猫,not-会说话的汤姆猫,0.000000,0.002111,0.747466,0.000000,0.250422
1,倒霉蛋涂涂,not-倒霉蛋涂涂,0.000000,1.000000,0.000000,0.000000,0.000000
2,僵小鱼,not-僵小鱼,0.000000,0.020833,0.000000,0.000000,0.979167
3,兔小贝,not-兔小贝,0.000000,0.000000,1.000000,0.000000,0.000000
4,动物兄弟,not-动物兄弟,0.000000,0.000000,0.000000,0.000000,1.000000
5,咖宝车神,not-咖宝车神,0.000000,1.000000,0.000000,0.000000,0.000000
6,咚咚玩具屋,not-咚咚玩具屋,0.000000,0.000000,0.000000,0.000000,1.000000
7,喜羊羊与灰太狼,not-喜羊羊与灰太狼,0.000000,0.008498,0.467369,0.000000,0.524133
8,大卫,not-大卫,0.000000,0.010311,0.458857,0.000000,0.530831
9,大头儿子小头爸爸,not-大头儿子小头爸爸,0.000000,0.000000,0.000000,0.000000,1.000000
